In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from time import sleep
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import datetime

In [2]:
import json

# Define the path to your config file
config_file_path = 'data/config.json'

# Function to read the config file and return the tc and password
def read_config(file_path):
    try:
        # Open the config file
        with open(file_path, 'r') as file:
            # Parse the JSON data
            config = json.load(file)
        
        # Extract the tc and password values
        tc = config.get('tc', 'default_tc_if_not_found')
        password = config.get('password', 'default_password_if_not_found')
        link = config.get('link', 'default_link_if_not_found')
        timeslot = config.get('timeslot', 'default_timeslot_if_not_found')
        
        return tc, password, link, timeslot
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None, None
    except json.JSONDecodeError:
        print("Error: There was an issue decoding the JSON file.")
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None

today_plus_3 = date.today() + datetime.timedelta(days=2)
today_plus_3 = today_plus_3.strftime("%d.%m.%Y")

# Use the function and print the results
tc, password, link, timeslot = read_config(config_file_path)
if tc and password:
    print(f"TC: {tc}")
    print(f"Password: {password}")
    print(f"Link: {link}")
    print(f"Timeslot: {timeslot}")
    print(f"For date: {today_plus_3}")

TC: 27754977334
Password: mrtsyhn123
Link: https://online.spor.istanbul/uyegiris
Timeslot: 08:00 - 09:00
For date: 15.02.2024


In [3]:
# driver execution and rules for better performance
chromeOptions = uc.ChromeOptions()
chromeOptions.add_argument('--blink-settings=imagesEnabled=false')
driver = uc.Chrome(options=chromeOptions)

driver.get(link) # siteye giriş
print("siteye girdi.")


# credential'ları gir
tcelem = driver.find_element(By.XPATH, "//*[@id='txtTCPasaport']").send_keys(tc)
passelem = driver.find_element(By.XPATH, "//*[@id='txtSifre']").send_keys(password)
benihatir = driver.find_element(By.XPATH, "//*[@id='cboxBeniHatirla']").click()
loginbutt = driver.find_element(By.XPATH, "//*[@id='btnGirisYap']").click()

print("credential'lar girildi.")

#CLICK ANYWHERE
# JavaScript to calculate the middle of the viewport and click
js_script = """
// Calculate middle of the viewport
const x = window.innerWidth / 2;
const y = window.innerHeight / 2;

// Create a mouse click event
const clickEvent = new MouseEvent('click', {
    'view': window,
    'bubbles': true,
    'cancelable': true,
    'clientX': x,
    'clientY': y
});

// Dispatch the event on the element at the calculated position
document.elementFromPoint(x, y).dispatchEvent(clickEvent);
"""

# Execute the JavaScript
driver.execute_script(js_script)

siteye girdi.
credential'lar girildi.


In [4]:
# kiralama hizmetleri kısmına tıklar
kiralamahizm = '//*[@id="form1"]/div[3]/div[2]/div/div/ul/li[3]/a'
driver.find_element(By.XPATH, kiralamahizm).click()


# BRANŞ seç
bransdropdown = driver.find_element(By.ID, "ddlBransFiltre")
selectbrans = Select(bransdropdown)
selectbrans.select_by_visible_text("TENİS")
sleep(1)

# tesis seç
tesisdropdown = driver.find_element(By.ID, "ddlTesisFiltre")
selecttesis = Select(tesisdropdown)
selecttesis.select_by_visible_text("MALTEPE SAHİL SPOR TESİSİ")
sleep(1)

# salon seç
salondd = driver.find_element(By.ID, "ddlSalonFiltre")
selectsalon = Select(salondd)
selectsalon.select_by_visible_text("KAPALI TENIS KORTU 3")

In [5]:
print(str(today_plus_3))

15.02.2024


In [7]:
# en aşağı scroll'la, bulsun diye
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# 3 gün sonrayı bulsun
date_containers = driver.find_elements(By.CLASS_NAME, 'panel-heading')
sleep(1)
target_date_container = None
for container in date_containers:
    if today_plus_3 in container.text:
        target_date_container = container.find_element(By.XPATH, './..')  # Navigate to parent container
        break

if target_date_container:
    # Within this container, find the '22:00 - 23:00' slot
    time_slots = target_date_container.find_elements(By.CLASS_NAME, 'lblStyle')
    sleep(1)
    target_time_slot = None
    for slot in time_slots:
        if timeslot in slot.text:
            target_time_slot = slot.find_element(By.XPATH, './..')  # Navigate to parent container of the slot
            break
    
    if target_time_slot:
        print("Found the target time slot container.")
        # Now, find and click the 'Rezervasyon' button within this container
        sleep(1)
        try:
            rezervasyon_button = target_time_slot.find_element(By.XPATH, ".//a[contains(text(), 'Rezervasyon')]")
            sleep(.1)
            rezervasyon_button.click()
            print("Clicked the 'Rezervasyon' button.")
        except Exception as e:
            print("Could not find or click the 'Rezervasyon' button:", e)
    else:
        print(f"Time slot {timeslot}] not found within the date container.")
else:
    print(f"Date {today_plus_3} not found.")

try:
    # Switch to the alert
    alert = Alert(driver)
    alert.accept()  # To accept the alert
    # alert.dismiss()  # To dismiss the alert, if it's a confirm dialog
except Exception as e:
    print("No JavaScript alert found:", e)
sleep(3)
kortkirala = '''//*[@id="rblKiralikTenisSatisTuru"]/tbody/tr[3]/td/label'''
driver.find_element(By.XPATH, kortkirala).click()
sleep(1)
kiraliksoz = '''//*[@id="pageContent_dvSepet"]/p[2]/span/label'''
driver.find_element(By.XPATH, kiraliksoz).click()
sleep(1)
sepete_ekle = '''//*[@id="pageContent_lbtnSepeteEkle"]'''
driver.find_element(By.XPATH, sepete_ekle).click()

Found the target time slot container.
Clicked the 'Rezervasyon' button.
